In [ ]:
import json
import folium
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import Polygon

In [ ]:
with open('./files/광주광역시_서구.json', 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

In [ ]:
import json
import folium

# 1. JSON 파일 열기
with open('./광주광역시_서구.json', 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

# 4. folium 지도 생성
#  m = folium.Map(location=[35.1445, 126.9156], tiles='cartodbpositron', zoom_start=13)
m = folium.Map(
    location=[35.1445, 126.9156],
    tiles='cartodbpositron',
    zoom_start=13,
    zoom_control=True,          # 줌 버튼 비활성화
    scrollWheelZoom=True,       # 마우스 휠 줌 비활성화
    dragging=True               # 드래그 이동 비활성화
)
# 5. GeoJson 추가 (툴팁에 동 이름 표시)
folium.GeoJson(
    geo_data,
    name='광주 서구 통합 동',
    tooltip=folium.GeoJsonTooltip(fields=['adm_nm'], aliases=['행정동:'])
).add_to(m)

folium.LayerControl().add_to(m)

# 6. 지도 출력
m



In [ ]:
for feature in geo_data["features"]:
    adm_nm = feature["properties"]["adm_nm"]
    dong_name = adm_nm.split()[-1]
    # print(dong_name)

In [ ]:
# 양동 = 양동, 양3동
# 농성동 = 농성1동, 농성2동
# 금호동 = 금호1동, 금호2동
# 화정동 = 화정2동, 화정3동, 화정4동
# 광천동
# 치평동
# 유덕동
# 상무1동
# 상무2동
# 서창동
# 풍암동
# 동천동

In [ ]:
case1 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 양동")
case2 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 양3동")

case1 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 농성1동")
case2 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 농성2동")

case1 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 금호1동")
case2 = next(f for f in geo_data["features"] if f["properties"]["adm_nm"] == "광주광역시 서구 금호2동")

# 위도, 경도 좌표를 2차원 리스트로 가져옴
coords1 = case1['geometry']['coordinates'][0]
coords2 = case2['geometry']['coordinates'][0]

# shapely의 Polygon으로 변환 (경도, 위도 순서로!)
poly1 = Polygon([(lon, lat) for lon, lat in coords1])
poly2 = Polygon([(lon, lat) for lon, lat in coords2])

# 병합 (union)
merged = unary_union([poly1, poly2])

# folium에서 사용하려면 다시 (위도, 경도)로 변환
merged_coords = [(lat, lon) for lon, lat in merged.exterior.coords]

# 지도 중심 설정
center_lat = sum([c[0] for c in merged_coords]) / len(merged_coords)
center_lon = sum([c[1] for c in merged_coords]) / len(merged_coords)

m = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles='cartodbpositron')

# 병합된 폴리곤 추가
folium.Polygon(
    locations=merged_coords,
    color='blue',
    weight=2,
    fill=True,
    fill_color='blue',
    fill_opacity=0.4
).add_to(m)

m

In [ ]:
import folium
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import json
from collections import defaultdict

# 행정동 그룹 정의
districts = {
    '양동': ['광주광역시 서구 양동', '광주광역시 서구 양3동'],
    '농성동': ['광주광역시 서구 농성1동', '광주광역시 서구 농성2동'],
    '금호동': ['광주광역시 서구 금호1동', '광주광역시 서구 금호2동'],
    '화정동': ['광주광역시 서구 화정1동','광주광역시 서구 화정2동', '광주광역시 서구 화정3동', '광주광역시 서구 화정4동'],
    '광천동': ['광주광역시 서구 광천동'],
    '치평동': ['광주광역시 서구 치평동'],
    '유덕동': ['광주광역시 서구 유덕동'],
    '상무1동': ['광주광역시 서구 상무1동'],
    '상무2동': ['광주광역시 서구 상무2동'],
    '서창동': ['광주광역시 서구 서창동'],
    '풍암동': ['광주광역시 서구 풍암동'],
    '동천동': ['광주광역시 서구 동천동'],
}

# 병합된 결과를 저장할 딕셔너리
merged_polygons = {}
geojson_features = []

# folium 지도 초기화용 변수
all_coords = []

for name, adm_list in districts.items():
    polygons = []

    for adm_nm in adm_list:
        feature = next((f for f in geo_data['features'] if f['properties']['adm_nm'] == adm_nm), None)
        if feature is None:
            print(f"❌ {adm_nm} 찾을 수 없음")
            continue

        # 좌표 구조 파악: Polygon 또는 MultiPolygon
        coords_group = feature['geometry']['coordinates']
        if feature['geometry']['type'] == 'Polygon':
            coords_group = [coords_group]  # Polygon은 바로 리스트로 변환

        for coords in coords_group:
            poly = Polygon([(lon, lat) for lon, lat in coords[0]])
            polygons.append(poly)

    if polygons:
        unioned = unary_union(polygons)
        merged_polygons[name] = unioned

        # GeoJSON feature로 저장
        if isinstance(unioned, Polygon):
            exterior = [[lon, lat] for lon, lat in unioned.exterior.coords]
            geometry = {
                "type": "Polygon",
                "coordinates": [exterior]
            }
            all_coords.extend(exterior)
        elif isinstance(unioned, MultiPolygon):
            multipoly_coords = [[[lon, lat] for lon, lat in poly.exterior.coords] for poly in unioned.geoms]
            geometry = {
                "type": "MultiPolygon",
                "coordinates": [[c] for c in multipoly_coords]
            }
            for poly in multipoly_coords:
                all_coords.extend(poly)

        geojson_features.append({
            "type": "Feature",
            "properties": {
                "adm_nm": name
            },
            "geometry": geometry
        })

# folium 지도 중심 계산
center_lat = sum([lat for lon, lat in all_coords]) / len(all_coords)
center_lon = sum([lon for lon, lat in all_coords]) / len(all_coords)

m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='cartodbpositron')

# 병합된 polygon들을 folium에 그림
for name, poly in merged_polygons.items():
    if isinstance(poly, Polygon):
        coords = [(lat, lon) for lon, lat in poly.exterior.coords]
        folium.Polygon(
            locations=coords,
            tooltip=name,
            color='blue',
            fill=True,
            fill_opacity=0.4
        ).add_to(m)
    elif isinstance(poly, MultiPolygon):
        for subpoly in poly.geoms:
            coords = [(lat, lon) for lon, lat in subpoly.exterior.coords]
            folium.Polygon(
                locations=coords,
                tooltip=name,
                color='blue',
                fill=True,
                fill_opacity=0.4
            ).add_to(m)

# GeoJSON 저장
merged_geojson = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open("merged_seogu_districts.geojson", "w", encoding='utf-8') as f:
    json.dump(merged_geojson, f, ensure_ascii=False, indent=2)

m  # 마지막에 지도를 출력


In [14]:
import folium
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import json
from collections import defaultdict

# 행정동 그룹 정의
districts = {
    '서구': ['광주광역시 서구 양동', '광주광역시 서구 양3동', '광주광역시 서구 농성1동', '광주광역시 서구 농성2동', '광주광역시 서구 금호1동', '광주광역시 서구 금호2동'
            '광주광역시 서구 화정1동','광주광역시 서구 화정2동', '광주광역시 서구 화정3동', '광주광역시 서구 화정4동','광주광역시 서구 광천동'
,'광주광역시 서구 치평동'
,'광주광역시 서구 유덕동'
,'광주광역시 서구 상무1동'
,'광주광역시 서구 상무2동'
,'광주광역시 서구 서창동'
,'광주광역시 서구 풍암동'
,'광주광역시 서구 동천동']
}

# 병합된 결과를 저장할 딕셔너리
merged_polygons = {}
geojson_features = []

# folium 지도 초기화용 변수
all_coords = []

for name, adm_list in districts.items():
    polygons = []

    for adm_nm in adm_list:
        feature = next((f for f in geo_data['features'] if f['properties']['adm_nm'] == adm_nm), None)
        if feature is None:
            print(f"❌ {adm_nm} 찾을 수 없음")
            continue

        # 좌표 구조 파악: Polygon 또는 MultiPolygon
        coords_group = feature['geometry']['coordinates']
        if feature['geometry']['type'] == 'Polygon':
            coords_group = [coords_group]  # Polygon은 바로 리스트로 변환

        for coords in coords_group:
            poly = Polygon([(lon, lat) for lon, lat in coords[0]])
            polygons.append(poly)

    if polygons:
        unioned = unary_union(polygons)
        merged_polygons[name] = unioned

        # GeoJSON feature로 저장
        if isinstance(unioned, Polygon):
            exterior = [[lon, lat] for lon, lat in unioned.exterior.coords]
            geometry = {
                "type": "Polygon",
                "coordinates": [exterior]
            }
            all_coords.extend(exterior)
        elif isinstance(unioned, MultiPolygon):
            multipoly_coords = [[[lon, lat] for lon, lat in poly.exterior.coords] for poly in unioned.geoms]
            geometry = {
                "type": "MultiPolygon",
                "coordinates": [[c] for c in multipoly_coords]
            }
            for poly in multipoly_coords:
                all_coords.extend(poly)

        geojson_features.append({
            "type": "Feature",
            "properties": {
                "adm_nm": name
            },
            "geometry": geometry
        })

# folium 지도 중심 계산
center_lat = sum([lat for lon, lat in all_coords]) / len(all_coords)
center_lon = sum([lon for lon, lat in all_coords]) / len(all_coords)

m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='cartodbpositron')

# 병합된 polygon들을 folium에 그림
for name, poly in merged_polygons.items():
    if isinstance(poly, Polygon):
        coords = [(lat, lon) for lon, lat in poly.exterior.coords]
        folium.Polygon(
            locations=coords,
            tooltip=name,
            color='blue',
            fill=True,
            fill_opacity=0.4
        ).add_to(m)
    elif isinstance(poly, MultiPolygon):
        for subpoly in poly.geoms:
            coords = [(lat, lon) for lon, lat in subpoly.exterior.coords]
            folium.Polygon(
                locations=coords,
                tooltip=name,
                color='blue',
                fill=True,
                fill_opacity=0.4
            ).add_to(m)

# GeoJSON 저장
merged_geojson = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open("test.geojson", "w", encoding='utf-8') as f:
    json.dump(merged_geojson, f, ensure_ascii=False, indent=2)

m  # 마지막에 지도를 출력


❌ 광주광역시 서구 금호2동광주광역시 서구 화정1동 찾을 수 없음


In [ ]:
geometry_count = sum(1 for feature in merged_geojson["features"] if "geometry" in feature)
print("geometry 개수:", geometry_count)